In [1]:
import requests
from bs4 import BeautifulSoup

raw = requests.get("https://search.naver.com/search.naver?where=news&query=직방")
html = BeautifulSoup(raw.text, "html.parser")

- 하지만 약간의 제안 사항이 있다.

1) API 호출이 일 25,000회로 제한되어 있다. 최대 한 번 호출에 100개의 검색을 가져 올 수 있다.

2) 네이버에서 데이터를 검색하면 뉴스에서만 약 43만건의 데이터가 검색된다. 하지만 최대 1000개만 API를 이용하여 가지고 올 수 있다.

​

In [2]:
raw

<Response [200]>

In [33]:
import requests
import json
import time   
import pandas as pd

url = 'https://openapi.naver.com/v1/search/news?'
clientId = Naver_News_clientId
clientSecret = Naver_News_clientId_clientSecret
header = {
    "X-Naver-Client-Id": clientId ,
    "X-Naver-Client-secret":clientSecret,
}

#start가 100에서 처음 등장하는 것이 start 1일때 newslist[99]임
#들어가야할 start 값 -> 1, 101, 201, 301, 401, 
for i in range(0, 10):
    start = 1
    while start <= 1000 : #start는 1000개가 최대
        time.sleep(0.1)
        params = {'query':"구글" ,'start':start, 'sort':'sim'}# 'display':100, 'start':start, 'sort':'sim'} #'sort':'date'} #날짜순 아닌 유사도 순
        r = requests.get(url, params = params, headers = header)
        time.sleep(0.1)
        j = r.json() 
        newslist = j['items'] 
        #df = df.append(newslist)
        start += 100
        time.sleep(0.1)
        #print(j)

    newslist.append(newslist)



In [34]:
newslist

[{'title': '공격적 M&amp;A &apos;<b>직방</b>&apos;, 적자전환해 영업손실 82억… 부채 6배 ↑',
  'originallink': 'http://moneys.mt.co.kr/news/mwView.php?no=2022040411368032267',
  'link': 'https://n.news.naver.com/mnews/article/417/0000800417?sid=101',
  'description': '4일 금융감독원 전자공시시스템에 <b>따르면</b> 지난해 <b>직방</b>은 매출(연결 기준)이 전년대비 21.9% 증가해 558억8000만원으로 집계됐다. <b>직방</b>이 매출 500억대를 기록한 건 창립 이래 처음이다. 지난 3년간 매출은... ',
  'pubDate': 'Mon, 04 Apr 2022 12:03:00 +0900'},
 {'title': '<b>직방</b>, &apos;연이은 M&amp;A 영향&apos; 1년만에 적자 전환…영업손실 82억원',
  'originallink': 'https://news.mtn.co.kr/news-detail/2022040414381439932',
  'link': 'https://news.mtn.co.kr/news-detail/2022040414381439932',
  'description': '4일 금융감독원 전자공시시스템에 <b>따르면</b> 지난해 <b>직방</b>은 연결기준 매출이 전년보다 21.9% 증가한 558억 8,000만원으로 집계됐다. <b>직방</b> 매출이 500억원을 넘어선 건 이번이 처음이다. 그러나 영업손실은 82억 3... ',
  'pubDate': 'Mon, 04 Apr 2022 14:48:00 +0900'},
 {'title': '“월세 선호도 2년 전과 비교해 두 배 이상 증가, 이유가 있었다”',
  'originallink': 'https://www.wikitree.co.kr/articles/788573',
  'link':

In [48]:
pd.DataFrame(newslist[0], index=[0])

,title,originallink,link,description,pubDate
0,"공격적 M&amp;A &apos;<b>직방</b>&apos;, 적자전환해 영업손실 ...",http://moneys.mt.co.kr/news/mwView.php?no=2022...,https://n.news.naver.com/mnews/article/417/000...,4일 금융감독원 전자공시시스템에 <b>따르면</b> 지난해 <b>직방</b>은 매출...,"Mon, 04 Apr 2022 12:03:00 +0900"


In [50]:

df_zigbang = pd.json_normalize( newslist[0] )


In [51]:
df_zigbang.tail()

,title,originallink,link,description,pubDate
0,"공격적 M&amp;A &apos;<b>직방</b>&apos;, 적자전환해 영업손실 ...",http://moneys.mt.co.kr/news/mwView.php?no=2022...,https://n.news.naver.com/mnews/article/417/000...,4일 금융감독원 전자공시시스템에 <b>따르면</b> 지난해 <b>직방</b>은 매출...,"Mon, 04 Apr 2022 12:03:00 +0900"


In [52]:
df_zigbang.pubDate = pd.to_datetime(df_zigbang.pubDate) 
df_zigbang.pubDate = df_zigbang.pubDate.dt.tz_localize(None)

In [105]:
df_zigbang['year-m'] = df_zigbang.pubDate.dt.strftime('%Y-%m')

In [106]:
#df_zigbang.pivot_table(index='year-m', aggfunc='count' )
df_zigbang.to_excel('naver_news_zigbang.xlsx')


### 네이버 뉴스 크롤러 

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
#from config import *

#[CODE 1]
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", clientId)
    req.add_header("X-Naver-Client-Secret", clientSecret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getNaverSearchResult(sNode, search_text, page_start, display):
    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

#[CODE 3]
def getPostData(post, jsonResult):
    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    #Tue, 14 Feb 2017 18:46:00 +0900
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'title':title, 'description': description,
                    'org_link':org_link, 'link': org_link, 
                    'pDate':pDate})
    return    

def main():

    jsonResult = []

    # 'news', 'blog', 'cafearticle'
    sNode = 'news'
    search_text = '구글'
    display_count = 100
    
    jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)
    
    while ((jsonSearch != None) and (jsonSearch['display'] != 0)):
        for post in jsonSearch['items']:
            getPostData(post, jsonResult)
        
        nStart = jsonSearch['start'] + jsonSearch['display']
        jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
    
    with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf8') as outfile:
        retJson = json.dumps(jsonResult,
                        indent=4, sort_keys=True,
                        ensure_ascii=False)
        outfile.write(retJson)
        
    print ('%s_naver_%s.json SAVED' % (search_text, sNode))

    
if __name__ == '__main__':
    main()

In [31]:
df = pd.read_json(r'/Users/reejungkim/Documents/Git/WebCrawler/naver_news.json')

In [32]:
df

,description,link,org_link,pDate,title
0,부동산 빅데이터 업체 &apos;<b>아실</b>&apos;에 <b>따르면</b> ...,http://www.mdilbo.com/detail/joFjOo/686725,http://www.mdilbo.com/detail/joFjOo/686725,2023-01-13 17:12:00,매매보단 안정적···지난해 물량 소진 &apos;변수&apos;
1,의견에 <b>따르면</b> 1조그루 정도의 더 나무를 심는다면 탄소 중립에 굉장히 ...,https://news.bbsi.co.kr/news/articleView.html?...,https://news.bbsi.co.kr/news/articleView.html?...,2023-01-13 16:50:00,&apos;부산을 바꾸자&apos;(2)-김동필 부산대 조경학과 교수
2,<b>아실에 따르면</b> 이날 개포동 전세 물건은 2477건으로 지난해(1007건...,https://www.mk.co.kr/article/10605595,https://www.mk.co.kr/article/10605595,2023-01-13 14:15:00,맹모들 슬슬 군침 흘릴까…학군지 공급 폭탄에 전셋값 줄하락
3,"자료에 <b>따르면</b> 2020년의 경우 부산시가 1위, 대구시가 5위였습니다....",https://dgmbc.com/article/HdwQIjrtD4FM,https://dgmbc.com/article/HdwQIjrtD4FM,2023-01-13 11:38:00,"[들어보니] 위협받는 보행권, 보행위험구역 | 빅벙커"
4,<b>아실에 따르면</b> 지난 3일 시장에 나와있는 서울 매물은 4만9774건이었...,http://www.smedaily.co.kr/news/articleView.htm...,http://www.smedaily.co.kr/news/articleView.htm...,2023-01-13 11:12:00,1.3대책에도 급매물 위주 거래 여전 여의도·목동 등 수억 하락 거래 속출
...,...,...,...,...,...
995,부동산 빅데이터 업체 <b>아실에 따르면</b> 이날 기준 서울 아파트 전세 매물은...,https://www.ajunews.com/view/20220928132351992,https://www.ajunews.com/view/20220928132351992,2022-09-28 13:26:00,매물 쌓이는 서울 아파트 전세시장…전세수급지수 &apos;수요&gt;공급&apos; 전환
996,부동산 빅데이터 업체 <b>아실에 따르면</b> 이날 기준 서울 아파트 전세 매물은...,https://www.chosun.com/economy/real_estate/202...,https://www.chosun.com/economy/real_estate/202...,2022-09-28 10:42:00,매물 쌓이는 전세시장… 민간 통계도 수요자 우위로 돌아서
997,<b>아실에 따르면</b> 27일 기준 서울 임대차 시장의 전세 물량은 3만8925...,http://www.queen.co.kr/news/articleView.html?i...,http://www.queen.co.kr/news/articleView.html?i...,2022-09-28 10:06:00,서울 아파트 전세 3년7개월만에 &apos;공급&gt;수요&apos; ... 깡통전...
998,28일 부동산 빅데이터 플랫폼 <b>아실에 따르면</b> 앞서 주거정책심의위원회가 ...,https://www.dailian.co.kr/news/view/1156762/?s...,https://www.dailian.co.kr/news/view/1156762/?s...,2022-09-28 06:10:00,살 사람 없다 vs 거래 살아날 수도…매물 거두는 집주인 &apos;동상이몽&apos;


In [163]:
df_zingbang = pd.read_json(r'/Users/reejungkim/Documents/Git/WebCrawler/아실에 따르면_naver_news.json')
df_zingbang.to_excel('아실에 따르면.xlsx')